In [1]:
import partridge as ptg
import datetime
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import networkx as nx
import pandas as pd
import os
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
from fiona import collection
from IPython.display import clear_output
import threading
import multiprocessing
%matplotlib inline

In [2]:
path=r"/Users/brianparker/Work/VRT_Transit1.zip"

In [3]:
service_ids_by_date=ptg.read_service_ids_by_date(path)

In [4]:
date=datetime.date(2018, 6, 4)

In [5]:
service_ids=service_ids_by_date[date]

In [6]:
feed=ptg.feed(path,view={
    'trips.txt':{
        'service_id':service_ids,
    },
})

In [7]:
df=feed.stops

In [8]:
df['Coordinates']=list(zip(df.stop_lat,df.stop_lon))

In [9]:
outfolder=r"/Users/brianparker/Work/Walksheds/"

In [10]:
allwalksheds=[]
allwalksheds=gpd.GeoDataFrame(allwalksheds)
d={}

In [11]:
def reader():
    for i in os.listdir(outfolder):
        d['{}'.format(i)]=gpd.read_file(outfolder+"\\"+i+"\\edges\\edges.shp")
        print('reading: '+i)
    for key,value in d.items():
        allwalksheds.append(value[:])
        print('appending: '+ key)
        return allwalksheds
    allwalksheds.plot()
    allwalksheds.to_file(r"N:\Planning - New File Structure\GIS\Data\Stops\allwalkshedsQtr.shp")

In [12]:
# reader()

In [13]:
# for key,value in d.items():
#     allwalksheds=allwalksheds.append(value[:])
#     print('appending: '+ key)

In [14]:
# allwalksheds.plot()

In [15]:
# test=ox.graph_to_gdfs(ox.graph_from_point([43.608106, -116.248038]),node_geometry=False,nodes=False)

In [16]:
def reader2(df):
    for index, row in df.iterrows():
        try:
#             clear_output(wait=True)
            stopname=row['stop_name']+"_"+row['stop_id']
            d['{}'.format(stopname)]=ox.graph_to_gdfs((ox.graph_from_point(row['Coordinates'],distance=200,network_type='walk',clean_periphery=False)),node_geometry=False,nodes=False)
            print('reading  {} ({} of {})'.format(row['stop_name'],(index)+1,len(df)))
        except Exception:
            print(stopname+" ERROR ERROR ERROR ERROR ERROR")
            continue
        

In [17]:
# downloadThreads = []             # a list of all the Thread objects
# for i in range(0, 1400, 100):    # loops 14 times, creates 14 threads
#     downloadThread = threading.Thread(target=reader2)
#     downloadThreads.append(downloadThread)
#     downloadThread.start()
# for downloadThread in downloadThreads:
#     downloadThread.join()
# print('Done.')

In [18]:
# reader2()

In [ ]:
# allwalksheds.to_file(r"/Users/brianparker/Work/Walksheds/allqtr.shp")

In [ ]:
# create as many processes as there are CPUs on your machine
num_processes = multiprocessing.cpu_count()

# calculate the chunk size as an integer
chunk_size = int(df.shape[0]/num_processes)

# this solution was reworked from the above link.
# will work even if the length of the dataframe is not evenly divisible by num_processes
chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]

/anaconda2/envs/py3mapping/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [ ]:
pool=multiprocessing.Pool(processes=num_processes)

reading  Broadway & Park (687 of 98)
reading  Collins @ V.A. Medical Center (197 of 98)
reading  12th & Sheridan (688 of 98)
reading  Five Mile & Irving (589 of 98)
reading  12th & Valley (689 of 98)
reading  Five Mile & Fairview (590 of 98)
reading  Holly & Florida (690 of 98)
reading  Elder @ Wells Fargo (491 of 98)
reading  Orchard & Bethel (99 of 98)
reading  Bird & Fern (691 of 98)
reading  Five Mile & Cory (591 of 98)
reading  Owyhee & Lemhi (492 of 98)
reading  12th & 4th (692 of 98)
reading  Apple & Lexington (393 of 98)
reading  Fort & 4th (198 of 98)
reading  Chrisway & Boise (394 of 98)
reading  2nd & 10th (693 of 98)
reading  Nez Perce & Owyhee (493 of 98)
reading  Fort & Washington (199 of 98)
reading  Apple & Old Saybrook (395 of 98)
reading  2nd & 3rd (694 of 98)
reading  Orchard & Emerald (100 of 98)
reading  University & Chrisway (1 of 98)
reading  50th & Sawyer (592 of 98)
reading  Apple & Ridgefield (396 of 98)
reading  Jefferson & Ave C (200 of 98)
reading  Nez Perc

In [ ]:
result=pool.map(reader2,chunks)

In [ ]:
for key,value in d.items():
    clear_output(wait=True)
    allwalksheds=allwalksheds.append(value[:])
    print(key)
    allwalksheds.plot()